### Import libraries

In [1]:
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm_notebook
import time
import json
import csv
import os

In [2]:
def save_url_file(url):
    urls = []
    root_url = "https://www.presidency.ucsb.edu"
    while True:
        req = requests.get(url)
        soup = BeautifulSoup(req.text, 'html.parser')
        all_urls = soup.find_all('td',class_='views-field-title')
        
        for u in all_urls:
            link = u.find('a')['href']
            urls.append(root_url + link)
               
        next_page = soup.find('a', {'title': 'Go to next page'})
        if next_page:
            url = root_url + next_page['href']
        else:
            break
        time.sleep(3)
        

    f = open('url.txt', 'w')
    f.write('\n'.join(urls))
    f.close()

In [3]:
save_url_file('https://www.presidency.ucsb.edu/advanced-search?field-keywords=&field-keywords2=&field-keywords3=&from%5Bdate%5D=01-01-1947&to%5Bdate%5D=03-22-2022&person2=&category2%5B%5D=45&category2%5B%5D=400&items_per_page=50')

In [10]:
def crawl_data(file):
    file1 = open(file , 'r')
    urls = file1.readlines()
    directory = "Data/"
    rows=[]
    index = 0
        
    for url in tqdm_notebook(urls):
        req = requests.get(url)
        soup = BeautifulSoup(req.text, 'html.parser')
        text = soup.find('div', class_= 'field-docs-content').text
        president = soup.find("h3", class_='diet-title').text
        categories = str([i.text for i in soup.findAll(attrs={"property": "rdfs:label skos:prefLabel", "typeof": "skos:Concept"})])
        date = soup.find("span", class_= 'date-display-single').text
        citation = soup.find("p", class_= 'ucsbapp_citation').text
        title = soup.find("div", class_= 'field-ds-doc-title').text.strip()
        rows.append([str(index)+".txt", president, categories, date, citation, title])
        
        with open(directory + "Text_Files/" + str(index)+".txt", 'w', encoding="utf-8") as f:
            f.write(text)
            
        time.sleep(3)
        index+= 1
        
    fields = ['Text', 'President', 'Categories', 'Date', 'Citation', 'Title']     
    with open(directory + 'metadata.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=",")
        writer.writerow(fields)
        writer.writerows(rows)

      
        

In [11]:
crawl_data('url.txt')

C:\Users\Nik\AppData\Local\Temp\ipykernel_14836\1646720538.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for url in tqdm_notebook(urls):


  0%|          | 0/82 [00:00<?, ?it/s]